In [1]:
import pandas as pd
import numpy as np

# Establecimientos

In [2]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Despliegue prueba\Establecimientos/"
dicc_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data/"

## Lectura de archivos

In [3]:
#### Lectura de archivos

#df_id: Módulo de identificación  
df_id = pd.read_excel(data_path+"Crudo/CENSO 2022/Capítulo 1.xlsx",header = 0)
#df_ib: Módulo identificación básica  
df_ib = pd.read_excel(data_path+"Crudo/CENSO 2022/Capítulo 2.xlsx",header = 0)
#df_po: Módulo de personal ocupado
df_po = pd.read_excel(data_path+"Crudo/CENSO 2022/Capítulo 3.xlsx",header = 0)
#df_cg: Módulo de costos y gastos
df_cg = pd.read_excel(data_path+"Crudo/CENSO 2022/Capítulo 4.xlsx",header = 0)
#df_ing: Módulo de ingresos
df_ing = pd.read_excel(data_path+"Crudo/CENSO 2022/Capítulo 5.xlsx",header = 0)
# df_res: Resultados de la entrevista
df_res = pd.read_excel(data_path+"Crudo/CENSO 2022/CE_REGISTROS_COLA_STATUS.xlsx",header = 0)

C:\Users\carlo\AppData\Roaming\Python\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
print("Dimensiones df resultados entrevista:",df_res.shape, "\n")
print("Dimensiones df identificación:",df_id.shape, "\n")
print("Dimensiones df info básica:",df_ib.shape, "\n")
print("Dimensiones df personal ocupado:",df_po.shape, "\n")
print("Dimensiones df costos y gastos:",df_cg.shape, "\n")
print("Dimensiones df ingresos:",df_ing.shape, "\n")

Dimensiones df resultados entrevista: (3888, 32) 

Dimensiones df identificación: (3514, 49) 

Dimensiones df info básica: (3514, 99) 

Dimensiones df personal ocupado: (3514, 81) 

Dimensiones df costos y gastos: (3514, 72) 

Dimensiones df ingresos: (3514, 26) 



## Filtrado de la información
1. Quedarse con las encuestas:  
    Status = Valido  
    Estado = Completa

In [5]:
df_res = df_res[df_res['STATUS']=='VALIDO']
df_res = df_res[df_res['ESTADO']=='Completa']

In [45]:
df = pd.merge(df_res, df_id, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_ib, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_po, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_cg, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_ing, on='UID_ENCUESTA', how='left')

C:\Users\carlo\AppData\Local\Temp\ipykernel_10900\1150504175.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'IMEI_x', 'ID_ENCUESTA_x', 'ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_po, on='UID_ENCUESTA', how='left')
C:\Users\carlo\AppData\Local\Temp\ipykernel_10900\1150504175.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'USUARIO_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_cg, on='UID_ENCUESTA', how='left')
C:\Users\carlo\AppData\Local\Temp\ipykernel_10900\1150504175.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'FECHACREACION_x', 'IMEI_x', 'ID_ENCUESTA_x', 'ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_ing, on='UID_ENCUESTA', how='left')


In [46]:
df.rename(columns={"CLASE_x": "CLASE",
                   "SALPOHTEMP":"SALTEMPH",
                   'SALPOMTEMP':"SALTEMPM"}, inplace=True)

ubi_dic = {"1. Establecimiento fijo (Local, tiendas, oficinas, consultorios, bodegas, parqueaderos)":"1. Establecimiento fijo",
           "3. Vivienda con actividad económica":"3. Vivienda con actividad económica visible",
           "2. Establecimiento semifijo (Puesto anclado al piso, caseta, kioscos, islas)":"2. Establecimiento semifijo"}
df['UBICA'] = df['UBICA'].replace(ubi_dic)

rut_dic = {'SÍ':1,
           'NO':2}
df['IDRUT'] = df['IDRUT'].replace(rut_dic)

cont_dic = {"4. No lleva registros":"4. No lleva registros",
           "3. Otro tipo de cuentas (libreta, cuaderno, Excel, caja registradora)":"3. Otro tipo de cuentas",
           "2. Libro de registro diario de operaciones":"2. Libro de registro de operaciones",
           "1. Estado de situación financiera o Estado de Resultados Integrales (PyG)":"1. Estado de Situación Financiera o Estado de Resultados Integrales (PyG)"}
df['REG_CONT'] = df['REG_CONT'].replace(cont_dic)


# Función para verificar si '1' está presente en la cadena
def contain_one(value):
    '''
    Crear un vector de 1's y 0's con el objetivo crear la variable de Cámara de Comercio
    '''
    if pd.isna(value):
        return np.nan
    return "Cámara de Comercio" if '1' in value.split(',') else 0

# Crear la nueva columna
df['CAMCOMER'] = df['CUAL_AUT'].apply(contain_one)

df.to_csv(data_path + "Intermedio/CENU.csv")


In [47]:
df['UID_ENCUESTA'][df['DPTO'].isna()]

4        4780793
5        4780796
6        4780797
7        4780798
8        4780799
          ...   
1852    10780439
1855    10481065
1858    10780426
1860    10779794
1869    10508062
Name: UID_ENCUESTA, Length: 303, dtype: int64

## Ajuste variables

In [48]:
# Definición labels y diccionarios
dic_depto = {5:"Antioquia",8:"Atlántico",11:"Bogotá, D. C.",13:"Bolívar",15:"Boyacá",17:"Caldas",18:"Caquetá",
             19:"Cauca",20:"Cesar",23:"Córdoba",25:"Cundinamarca",27:"Chocó",41:"Huila",44:"La Guajira",47:"Magdalena",
             50:"Meta",52:"Nariño",54:"Norte De Santander",63:"Quindío",66:"Risaralda",68:"Santander",70:"Sucre",73:"Tolima",
             76:"Valle Del Cauca",88:"San Andrés"}

dic_sector = {2:"Industria",3:"Comercio",4:"Servicios"}

dic_rut = {1:"Sí",0:"No"}

dic_g12 = {3:"Industria",
           4:"Construcción",
           5:"Comercio",
           6:"Transporte y almacenamiento",
           7:"Alojamiento/servicios de comida",
           8:"Información y comunicaciones",
           9:"Actividades inmobiliarias",
           10:"Educación",
           11:"Actividades de salud humana",
           12:"Actividades artísticas",
}

rel_reg_con= pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="REG_CONT")
rel_ubica= pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="UBICA")
rel_cam = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="CAM_COMER")

### Ingresos

In [83]:
df['ING_TOTAL'].describe()

count    1.114000e+03
mean     1.078777e+08
std      6.747386e+08
min      0.000000e+00
25%      4.000000e+06
50%      1.500000e+07
75%      5.000000e+07
max      1.293199e+10
Name: ING_TOTAL, dtype: float64

In [67]:
#vars_ings = ["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN","ING_DON","ING_CONSTRUC","ING_ALQME"]
(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1)).value_counts()

True     1069
False     910
dtype: int64

In [68]:
ids = (df['UID_ENCUESTA'][((df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']]==99).any(axis = 1))]).tolist()

In [69]:
print("Número de NA's de la var. Ingreso cuando la suma de las individuales no coincide:")
df['ING_TOTAL'][~(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1))].isna().sum()
# 865 datos son NA y el resto pueden ser otro ingresos no contemplados al momento del Operativo

Número de NA's de la var. Ingreso cuando la suma de las individuales no coincide:


865

In [70]:
ids = df['UID_ENCUESTA'][~(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1)) & ~(df['ING_TOTAL'].isna())]

In [71]:
df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS','ING_TOTAL']][df['UID_ENCUESTA'].isin(ids)]

,ING_IND,ING_MAQUIL,ING_MMTO,ING_COM,ING_SERV,ING_OTROPER,ING_FINAN,ING_OTROS,ING_TOTAL
14,0.0,0.0,0.0,99.0,10000000.0,100000.0,NaN,NaN,1.010000e+07
59,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
179,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
218,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
441,80000000.0,0.0,400000.0,3000000.0,99.0,0.0,NaN,NaN,8.340000e+07
444,99.0,0.0,0.0,99.0,20000000.0,0.0,NaN,NaN,2.000000e+07
513,99.0,0.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
516,99.0,0.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
608,99.0,0.0,0.0,99.0,99.0,0.0,NaN,NaN,0.000000e+00
928,0.0,0.0,0.0,0.0,2.0,0.0,NaN,NaN,1.000000e+10


In [72]:
np.sum([1,1,np.nan])

nan

### Costos y Gastos

#### Gastos

In [73]:
(df['GTOTAL_ANO'] == df[["GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]].sum(axis=1)).value_counts()

False    1144
True      835
dtype: int64

In [74]:
df[['GTOTAL_ANO',"GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]][(df[['GTOTAL_ANO',"GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]]==99).any(axis=1)]

,GTOTAL_ANO,GARRIEN_ANO,GENERGIA_ANO,GCOMUN_ANO,GACUED_ANO,GCOMB_ANO,GMANT_ANO,GOTROS_ANO


In [75]:
(df['GTOTAL']==99).value_counts()

False    1979
Name: GTOTAL, dtype: int64

In [76]:
print("Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:")
df['GTOTAL_ANO'][~(df['GTOTAL_ANO'] == df[["GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]].sum(axis=1))].isna().sum()
# El total de datos que no coinciden son NA's

Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:


1144

In [77]:
(df['GTOTAL_ANOV'] == df[["GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]].sum(axis=1)).value_counts()

False    1726
True      253
dtype: int64

In [78]:
df[['GTOTAL_ANOV',"GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]][(df[['GTOTAL_ANOV',"GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]]==99).any(axis=1)]

,GTOTAL_ANOV,GARRIEN_ANOV,GENERGIA_ANOV,GCOMUN_ANOV,GACUED_ANOV,GCOMB_ANOV,GMANT_ANOV,GOTROS_ANOV


In [79]:
print("Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:")
df['GTOTAL_ANOV'][~(df['GTOTAL_ANOV'] == df[["GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]].sum(axis=1))].isna().sum()
# El total de datos que no coinciden son NA's

Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:


1726

In [49]:
# De esta manera se suman los gastos de estableicmientos y viviendas en una sola variable
df['GASTOS'] =  df[['GTOTAL_ANO','GTOTAL_ANOV']].sum(axis=1)
# Dejar NA's en función de las variables originales

In [50]:
df.loc[(df['GTOTAL_ANOV'].isna()) & (df['GTOTAL_ANO'].isna()), 'GASTOS'] = np.nan

#### Costos

In [82]:
(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1)).value_counts()

True     1070
False     909
dtype: int64

In [83]:
df[['COSTO_TOTAL','COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']][(df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']]==99).any(axis=1)]

,COSTO_TOTAL,COSTO_FINAN,COSTO_HONO,COSTO_MAQ,COSTO_MMTO,COSTO_MPE,COSTO_PUB,COSTO_SERV,COSTO_TRANS,COSTO_VTA
59,0.0,0.0,99.0,99.0,0.0,99.0,99.0,99.0,99.0,99.0
192,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0
513,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,0.0,99.0
515,0.0,0.0,0.0,99.0,0.0,99.0,0.0,0.0,99.0,99.0
608,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,99.0,99.0
609,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0,99.0
653,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,99.0,99.0
844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0
1116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,99.0
1146,9000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000000.0,99.0


In [84]:
print("Número de NA's de la var. Costo cuando la suma de las individuales no coincide:")
df['COSTO_TOTAL'][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))].isna().sum()
# 866 de 909 datos que no coinciden son NA's

Número de NA's de la var. Costo cuando la suma de las individuales no coincide:


866

In [85]:
(df['COSTO_TOTAL'][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))]).value_counts()

0.0           37
9000000.0      1
22000000.0     1
800000.0       1
3000000.0      1
96.0           1
600000.0       1
Name: COSTO_TOTAL, dtype: int64

In [86]:
(df[['COSTO_TOTAL','COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))])
# El valor restante de valores que no coinciden se debe a los 99 que presentan algunas de las variables individuales

,COSTO_TOTAL,COSTO_FINAN,COSTO_HONO,COSTO_MAQ,COSTO_MMTO,COSTO_MPE,COSTO_PUB,COSTO_SERV,COSTO_TRANS,COSTO_VTA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0
1976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Construcción de la variable de costos y gastos agregados

In [51]:
df['CYG'] = df[['COSTO_TOTAL','GASTOS']].sum(axis=1)

In [54]:
df[['COSTO_TOTAL','GASTOS','CYG']]

,COSTO_TOTAL,GASTOS,CYG
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,60000000.0,168720000.0,228720000.0
3,84000000.0,15840000.0,99840000.0
4,NaN,NaN,NaN
...,...,...,...
1974,12000000.0,1110000.0,13110000.0
1975,1000000.0,2400000.0,3400000.0
1976,NaN,NaN,NaN
1977,163704520.0,48840000.0,212544520.0


In [53]:
df.loc[(df['GASTOS'].isna()) & (df['COSTO_TOTAL'].isna()), 'CYG'] = np.nan

### Personal

In [90]:
df['POTTOT'].describe()

count    1130.000000
mean        3.197345
std        11.211273
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       322.000000
Name: POTTOT, dtype: float64

### Remuneración Total

In [91]:
df['UID_ENCUESTA'][(df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALTEMPH','SALTEMPM']]==99).any(axis=1)]

35          70
399    4807221
Name: UID_ENCUESTA, dtype: int64

In [55]:
df['REMUNERACION_TOTAL'] = (df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALTEMPH','SALTEMPM']].replace(99,np.nan).sum(axis=1))*df['MESOPA']

In [91]:
df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']][(df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']]==99).any(axis=1)]

KeyError: "['SALPOHTEMP', 'SALPOMTEMP'] not in index"

In [21]:
df['REMUNERACION_TOTAL'].describe()

count    1.155000e+03
mean     1.299048e+07
std      7.629155e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      6.500000e+06
max      1.582000e+09
Name: REMUNERACION_TOTAL, dtype: float64

### Departamento

In [25]:
df[df['DPTO'].isna()].head()

,Unnamed: 0,ID_x,ACER_x,AG_x,AU,C_POB_x,CANT_EDIF,CANT_UNIDAD,CLASE,CO_x,...,PART_OTROPER,PART_SERV,RINGCEROJUST,RVAGRE,USUARIO,VAGRE,CAMCOMER,REMUNERACION_TOTAL,GASTOS,CYG
4,8,561,99,962426,1,0.0,1,1,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,11,564,99,962405,1,0.0,1,1,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12,565,99,962405,1,0.0,1,1,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,13,566,99,962405,1,0.0,1,1,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,14,567,99,962413,1,0.0,1,1,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# Mantener sólo los departamentos que se encuentran en EMICRON
df['DPTO'].isin(dic_depto.keys()).value_counts()

False    1287
True      692
Name: DPTO, dtype: int64

### Ciudad Principal o Área Metropolitana  
Ciudades Principales y áreas metropolitanas:  
- 11001: Bogotá  
- 5001: Medellín  
    - Valle de Aburra:  
    - 5079: Barbosa  
    - 5088: Bello  
    - 5129: Caldas  
    - 5212: Copacabana  
    - 5266: Envigado  
    - 5308: Girardota  
    - 5360: Itagui  
    - 5380: La Estrella  
    - 5631: Sabaneta  
- 76001: Cali  
    - 76892: Yumbo
- 8001: Barranquilla
    - 8758: Soledad
- 68001: Bucaramanga
    - 68276: Floridablanca
    - 68307: Girón
    - 68547: Piedecuesta
- 17001: Manizales
    - 17873: Villamaría
- 52001: Pasto
- 66001: Pereira
    - 66170: Dosquebradas
    - 66400: La Virginia
- 73001: Ibagué
- 54001: Cúcuta
    - 54874: Villa del Rosario
    - 54405: Los Patios
    - 54261: El Zulia
- 50001: Villavicencio
- 23001: Montería
- 13001: Cartagena
- 15001: Tunja
- 18001: Florencia
- 19001: Popayán
- 20001: Valledupar
- 27001: Quibdó
- 41001: Neiva
- 44001: Riohacha
- 47001: Santa Marta
- 63001: Armenia
- 70001: Sincelejo
- 88001: San Andrés    

In [58]:
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,	8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,	54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	70001,	88001, 1]

In [59]:
df['CPAM'] = (df['MPIO'].isin(cpam) & df['CLASE']==1).apply(lambda x: int(x))

In [170]:
df[['MPIO','CLASE','CPAM']]

,MPIO,CLASE,CPAM
0,1.0,1,1
1,1.0,1,1
2,433.0,1,0
3,1.0,1,1
4,NaN,1,0
...,...,...,...
1974,1.0,1,1
1975,1.0,1,1
1976,1.0,1,1
1977,1.0,1,1


### Actividad Económica

In [60]:
rel_g12 = pd.read_excel(R"D:\DANE - Contrato\2024\Operativo Barrido\Data\Diccionario.xlsx", sheet_name="GRUPOS12")

In [61]:
def extract_first_two_digits(x):
    '''
    Definición de una función para extraer los primeros dos dígitos del CIIU para obtener la división
    x: Columna a la que se le aplicará la columna 
    '''
    try:
        # Convert x to string, split at decimal point if present, and extract first two characters
        first_two_digits_str = str(x).split('.')[0][:2]
        # Convert the extracted string to an integer
        first_two_digits_int = int(first_two_digits_str)
        return first_two_digits_int
    except (ValueError, TypeError):
        # Return NaN (or any other value you prefer) if conversion fails
        return np.nan

# Aplicar  the function to the 'CIIU_3' column
df['Division'] = df['CIIUSELECCIONADO'].apply(extract_first_two_digits)


In [101]:
df.shape

(1979, 359)

In [62]:
df = pd.merge(df,rel_g12,on='Division',how="left")

In [63]:
df[['GRUPOS12','Division','CIIUSELECCIONADO']]

,GRUPOS12,Division,CIIUSELECCIONADO
0,4.0,43.0,4311
1,6.0,49.0,4922
2,5.0,47.0,4711
3,5.0,47.0,4719
4,NaN,NaN,NaN
...,...,...,...
1974,5.0,47.0,4711
1975,9.0,77.0,7710
1976,7.0,56.0,5611
1977,11.0,88.0,8810


### RUT

In [64]:
df['IDRUT']

0       1.0
1       1.0
2       1.0
3       1.0
4       NaN
       ... 
1974    1.0
1975    1.0
1976    2.0
1977    1.0
1978    1.0
Name: IDRUT, Length: 1979, dtype: float64

### Cámara de Comercio

In [42]:
# Función para verificar si '1' está presente en la cadena
def contain_one(value):
    '''
    Crear un vector de 1's y 0's con el objetivo crear la variable de Cámara de Comercio
    '''
    if pd.isna(value):
        return 0
    return 1 if '1' in value.split(',') else 0

# Crear la nueva columna
df['CAMCOMER'] = df['CUAL_AUT'].apply(contain_one)

In [43]:
df['CAMCOMER'] = df['CAMCOMER'].map(dict(zip(rel_cam['CAMCOMER'], rel_cam['Codigo'])))

In [65]:
df['CAMCOMER']

0                      NaN
1                      NaN
2       Cámara de Comercio
3       Cámara de Comercio
4                      NaN
               ...        
1974                   NaN
1975    Cámara de Comercio
1976                   NaN
1977    Cámara de Comercio
1978    Cámara de Comercio
Name: CAMCOMER, Length: 1979, dtype: object

### Registros Contables

In [66]:
df['REG_CONT'] = df['REG_CONT'].map(dict(zip(rel_reg_con['REG_CONT'], rel_reg_con['Codigo'])))

In [67]:
df['REG_CONT'].value_counts()

4.0    623
3.0    513
2.0    262
1.0    182
Name: REG_CONT, dtype: int64

### Antigüedad

In [68]:
df['ANIOS_OPERACION'] = 2024 - df['IDAIO']

### Ubicación

In [69]:
df['UBICA'] = df['UBICA'].map(dict(zip(rel_ubica['UBICA'], rel_ubica['Codigo'])))

In [72]:
df['UBICA'].value_counts()

1.0    1202
3.0     393
2.0      81
Name: UBICA, dtype: int64

## Creación Variables

In [74]:
df['VA'] = df['ING_TOTAL'] - df['CYG']
df['C_V'] = df['CYG'] / (df['ING_TOTAL'] + 1)

## Selección Variables

In [75]:
var_estab = ['UID_ENCUESTA','ING_TOTAL','CYG','POTTOT','REMUNERACION_TOTAL','DPTO','CPAM','GRUPOS12','UBICA','IDRUT','CAMCOMER','REG_CONT','ANIOS_OPERACION','VA','C_V'] 

var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','ANOS_ESPACIO','HOMBRE']

In [76]:
df['UBICA'].value_counts()

1.0    1202
3.0     393
2.0      81
Name: UBICA, dtype: int64

In [93]:
df_final = df[var_estab]

In [81]:
df_final.head()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,1.0,NaN,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,1472675,150000000.0,228720000.0,7.0,15528000.0,13.0,0,5.0,1.0,1.0,Cámara de Comercio,3.0,5.0,-78720000.0,1.524800
3,4264925,180000000.0,99840000.0,4.0,11136000.0,91.0,1,5.0,1.0,1.0,Cámara de Comercio,2.0,27.0,80160000.0,0.554667
4,4780793,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtrado de unidades

In [94]:
# Seleccionar unidades económicas ubicadas en vivienda o establecimiento fijo (1 y 3)
df_final = df_final[df_final['UBICA'].isin([1,3])]

In [95]:
# Departamentos con los que se entrenó el modelo
depto_emicron = [8, 13, 44, 47, 70, 20, 25, 11, 18, 41, 15, 68, 54, 76, 52, 19, 17, 63, 73, 66,  5, 23, 27, 50, 88]
df_final = df_final[df_final['DPTO'].isin(depto_emicron)]

In [90]:
df['CAMCOMER'] = df['CAMCOMER'].fillna(0)

In [96]:
id_ig = df_final['UID_ENCUESTA'][((df_final.isna()) | (df_final==99)).any(axis=1)]

In [97]:
len(id_ig)

210

In [99]:
663-210

453

In [223]:
df_final['CAMCOMER'].map({'Cámara de Comercio':1})

0       NaN
1       NaN
2       1.0
14      NaN
15      1.0
       ... 
1925    NaN
1931    1.0
1933    1.0
1954    1.0
1968    NaN
Name: CAMCOMER, Length: 663, dtype: float64

In [128]:
id_na = df_final['UID_ENCUESTA'][df_final.isna().any(axis=1)]

In [28]:
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,
        8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,
        54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	
        70001,	88001]

In [130]:
id_na = df_final['UID_ENCUESTA'][((df_final.isna()) | (df_final==99)).any(axis=1)]

In [132]:
df_final.head()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,SÍ,0,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,SÍ,0,NaN,NaN,NaN,NaN
2,1472675,150000000.0,2.287200e+08,7.0,15528000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-7.872000e+07,1.524800
14,43,10100000.0,2.000000e+09,1.0,0.0,11.0,1,5.0,1.0,NO,0,1.0,24.0,-1.989900e+09,198.019782
15,45,1000005.0,1.000050e+05,12.0,48000000.0,11.0,1,7.0,1.0,SÍ,1,1.0,24.0,9.000000e+05,0.100004


In [120]:
# Descripción variables no responde
(df_final==99).sum()

UID_ENCUESTA          0
ING_TOTAL             0
CYG                   0
POTTOT                0
REMUNERACION_TOTAL    0
DPTO                  0
CPAM                  0
GRUPOS12              0
UBICA                 0
IDRUT                 0
CAMCOMER              0
REG_CONT              0
ANIOS_OPERACION       0
VA                    0
C_V                   0
dtype: int64

In [147]:
(df_final['UID_ENCUESTA'].isin(ids))

0       False
1       False
2       False
14       True
15      False
        ...  
1925    False
1931    False
1933    False
1954    False
1968    False
Name: UID_ENCUESTA, Length: 663, dtype: bool

In [148]:
df_final.head(14)

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,SÍ,0,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,SÍ,0,NaN,NaN,NaN,NaN
2,1472675,150000000.0,2.287200e+08,7.0,15528000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-7.872000e+07,1.524800
14,43,10100000.0,2.000000e+09,1.0,0.0,11.0,1,5.0,1.0,NO,0,1.0,24.0,-1.989900e+09,198.019782
15,45,1000005.0,1.000050e+05,12.0,48000000.0,11.0,1,7.0,1.0,SÍ,1,1.0,24.0,9.000000e+05,0.100004
16,1463212,500000.0,1.720000e+06,1.0,0.0,13.0,0,3.0,1.0,NO,0,4.0,14.0,-1.220000e+06,3.439993
29,1466730,144000000.0,1.087050e+08,3.0,0.0,13.0,0,5.0,3.0,NO,0,3.0,8.0,3.529500e+07,0.754896
30,1469310,9600000.0,4.760000e+06,1.0,0.0,13.0,0,12.0,1.0,NO,1,4.0,15.0,4.840000e+06,0.495833
31,1469318,54000000.0,6.046800e+07,4.0,0.0,13.0,0,5.0,1.0,SÍ,1,3.0,24.0,-6.468000e+06,1.119778
32,1469412,102000000.0,1.482560e+08,2.0,4200000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-4.625600e+07,1.453490


In [154]:
df_final['UID_ENCUESTA'][(df_final['UID_ENCUESTA'].isin(ids))]

14     43
218    26
Name: UID_ENCUESTA, dtype: int64

In [160]:
df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']][df['UID_ENCUESTA']==70]

,SALFIJOH,SALFIJOM,SALPERMH,SALPERMM,SALPOHTEMP,SALPOMTEMP,MESOPA
35,0.0,0.0,99.0,0.0,0.0,0.0,1.0


In [167]:
df_final[df_final['UID_ENCUESTA']==399]

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V


In [158]:
df_final['UID_ENCUESTA'][df_final['REMUNERACION_TOTAL']==99]

35    70
Name: UID_ENCUESTA, dtype: int64

In [179]:
df['POTTOT'].describe()

count    1130.000000
mean        3.197345
std        11.211273
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       322.000000
Name: POTTOT, dtype: float64

In [180]:
df['POTTOT']

0        1.0
1        NaN
2        7.0
3        4.0
4        NaN
        ... 
1974     1.0
1975     2.0
1976     4.0
1977    17.0
1978     NaN
Name: POTTOT, Length: 1979, dtype: float64

In [133]:
df['IDRUT'].map({"SÍ":1,
                 "NO":0})

0       1.0
1       1.0
2       1.0
3       1.0
4       NaN
       ... 
1974    1.0
1975    1.0
1976    0.0
1977    1.0
1978    1.0
Name: IDRUT, Length: 1979, dtype: float64

# Establecimientos: Simulación

In [100]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Despliegue prueba\Establecimientos/"
dicc_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data/"

## Lectura de archivos

In [101]:
#### Lectura de archivos

#df_id: Módulo de identificación  
df_id = pd.read_excel(data_path+"Crudo/2024/MOD1-UBICA.xlsx",header = 0)
#df_ib: Módulo identificación básica  
df_ib = pd.read_excel(data_path+"Crudo/2024/MOD2-ACTE.xlsx",header = 0)
#df_po: Módulo de personal ocupado
df_po = pd.read_excel(data_path+"Crudo/2024/MOD3-POC.xlsx",header = 0)
#df_cg: Módulo de costos y gastos
df_cg = pd.read_excel(data_path+"Crudo/2024/MOD4-CYG.xlsx",header = 0)
#df_ing: Módulo de ingresos
df_ing = pd.read_excel(data_path+"Crudo/2024/MOD5-ING.xlsx",header = 0)
# df_res: Resultados de la entrevista
#df_res = pd.read_excel(data_path+"CENSO 2022/CE_REGISTROS_COLA_STATUS.xlsx",header = 0)

In [102]:
#print("Dimensiones df resultados entrevista:",df_res.shape, "\n")
print("Dimensiones df identificación:",df_id.shape, "\n")
print("Dimensiones df info básica:",df_ib.shape, "\n")
print("Dimensiones df personal ocupado:",df_po.shape, "\n")
print("Dimensiones df costos y gastos:",df_cg.shape, "\n")
print("Dimensiones df ingresos:",df_ing.shape, "\n")

Dimensiones df identificación: (300, 11) 

Dimensiones df info básica: (300, 28) 

Dimensiones df personal ocupado: (300, 73) 

Dimensiones df costos y gastos: (300, 30) 

Dimensiones df ingresos: (300, 13) 



In [103]:
#df = pd.merge(df_res, df_id, on='UID_ENCUESTA', how='left')
df = pd.merge(df_id, df_ib, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_po, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_cg, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_ing, on='UID_ENCUESTA', how='left')

In [104]:
#df.rename(columns={"CLASE_x": "CLASE"}, inplace=True)
df['GTOTAL_ANOV'] = 1
df['CIIUSELECCIONADO'] = 1889
df.to_csv(data_path + "Intermedio/CENU_SIMULADO.csv")

## Ajuste variables

In [105]:
# Definición labels y diccionarios
dic_depto = {5:"Antioquia",8:"Atlántico",11:"Bogotá, D. C.",13:"Bolívar",15:"Boyacá",17:"Caldas",18:"Caquetá",
             19:"Cauca",20:"Cesar",23:"Córdoba",25:"Cundinamarca",27:"Chocó",41:"Huila",44:"La Guajira",47:"Magdalena",
             50:"Meta",52:"Nariño",54:"Norte De Santander",63:"Quindío",66:"Risaralda",68:"Santander",70:"Sucre",73:"Tolima",
             76:"Valle Del Cauca",88:"San Andrés"}

dic_sector = {2:"Industria",3:"Comercio",4:"Servicios"}

dic_rut = {1:"Sí",0:"No"}

dic_g12 = {3:"Industria",
           4:"Construcción",
           5:"Comercio",
           6:"Transporte y almacenamiento",
           7:"Alojamiento/servicios de comida",
           8:"Información y comunicaciones",
           9:"Actividades inmobiliarias",
           10:"Educación",
           11:"Actividades de salud humana",
           12:"Actividades artísticas",
}

rel_reg_con= pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="REG_CONT")
rel_ubica= pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="UBICA")
rel_g12 = pd.read_excel(R"D:\DANE - Contrato\2024\Operativo Barrido\Data\Diccionario.xlsx", sheet_name="GRUPOS12")

### Costos y Gastos

#### Gastos

In [106]:
# De esta manera se suman los gastos de estableicmientos y viviendas en una sola variable
df['GASTOS'] =  df[['GTOTAL_ANO','GTOTAL_ANOV']].sum(axis=1)
# Dejar NA's en función de las variables originales

In [107]:
df.loc[(df['GTOTAL_ANOV'].isna()) & (df['GTOTAL_ANO'].isna()), 'GASTOS'] = np.nan

#### Construcción de la variables de costos y gastos agregados

In [108]:
df['CYG'] = df[['COSTO_TOTAL','GASTOS']].sum(axis=1)

In [109]:
df[['COSTO_TOTAL','GASTOS','CYG']]

,COSTO_TOTAL,GASTOS,CYG
0,1.368480e+08,3.186625e+07,1.687142e+08
1,1.024033e+08,5.430252e+07,1.567058e+08
2,1.094108e+08,4.101737e+07,1.504281e+08
3,8.942701e+07,2.500911e+07,1.144361e+08
4,1.007306e+08,3.334200e+07,1.340726e+08
...,...,...,...
295,1.748346e+08,3.346614e+07,2.083007e+08
296,1.251210e+08,2.579219e+07,1.509132e+08
297,1.073023e+08,2.554715e+07,1.328495e+08
298,5.307983e+07,5.292526e+07,1.060051e+08


In [110]:
df.loc[(df['GASTOS'].isna()) & (df['COSTO_TOTAL'].isna()), 'CYG'] = np.nan

### Remuneración Total

In [111]:
df['REMUNERACION_TOTAL'] = (df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALTEMPH','SALTEMPM']].replace(99,np.nan).sum(axis=1))*df['MESOPA']

### Ciudad Principal o Área Metropolitana  
Ciudades Principales y áreas metropolitanas:  
- 11001: Bogotá  
- 5001: Medellín  
    - Valle de Aburra:  
    - 5079: Barbosa  
    - 5088: Bello  
    - 5129: Caldas  
    - 5212: Copacabana  
    - 5266: Envigado  
    - 5308: Girardota  
    - 5360: Itagui  
    - 5380: La Estrella  
    - 5631: Sabaneta  
- 76001: Cali  
    - 76892: Yumbo
- 8001: Barranquilla
    - 8758: Soledad
- 68001: Bucaramanga
    - 68276: Floridablanca
    - 68307: Girón
    - 68547: Piedecuesta
- 17001: Manizales
    - 17873: Villamaría
- 52001: Pasto
- 66001: Pereira
    - 66170: Dosquebradas
    - 66400: La Virginia
- 73001: Ibagué
- 54001: Cúcuta
    - 54874: Villa del Rosario
    - 54405: Los Patios
    - 54261: El Zulia
- 50001: Villavicencio
- 23001: Montería
- 13001: Cartagena
- 15001: Tunja
- 18001: Florencia
- 19001: Popayán
- 20001: Valledupar
- 27001: Quibdó
- 41001: Neiva
- 44001: Riohacha
- 47001: Santa Marta
- 63001: Armenia
- 70001: Sincelejo
- 88001: San Andrés    

In [112]:
cpam = [11001,5001,5079,5088,5129,5212,5266,5308,5360,5380,5631,76001,76892,8001,8758,68001,68276,68307,68547,17001,17873,52001,66001,66170,66400,73001,54001,54874,54405,54261,50001,23001,13001,15001,18001,19001,20001,27001,41001,44001,47001,63001,70001,88001]

In [113]:
df['MPIO'] = pd.to_numeric(df['MPIO'], errors= 'coerce')
df['CPAM'] = (df['MPIO'].isin(cpam) & df['CLASE']==1).apply(lambda x: int(x))

### Actividad Económica

In [115]:
def extract_first_two_digits(x):
    '''
    Definición de una función para extraer los primeros dos dígitos del CIIU para obtener la división
    x: Columna a la que se le aplicará la columna 
    '''
    try:
        # Convert x to string, split at decimal point if present, and extract first two characters
        first_two_digits_str = str(x).split('.')[0][:2]
        # Convert the extracted string to an integer
        first_two_digits_int = int(first_two_digits_str)
        return first_two_digits_int
    except (ValueError, TypeError):
        # Return NaN (or any other value you prefer) if conversion fails
        return np.nan

# Aplicar  the function to the 'CIIU_3' column
df['Division'] = df['CIIUSELECCIONADO'].apply(extract_first_two_digits)


In [116]:
df.shape

(300, 158)

In [117]:
df = pd.merge(df,rel_g12,on='Division',how="left")

In [118]:
df[['GRUPOS12','Division','CIIUSELECCIONADO']]

,GRUPOS12,Division,CIIUSELECCIONADO
0,3,18,1889
1,3,18,1889
2,3,18,1889
3,3,18,1889
4,3,18,1889
...,...,...,...
295,3,18,1889
296,3,18,1889
297,3,18,1889
298,3,18,1889


### RUT

In [119]:
df['IDRUT']

0      2
1      2
2      1
3      2
4      2
      ..
295    2
296    1
297    1
298    2
299    2
Name: IDRUT, Length: 300, dtype: int64

### Registros Contables

In [120]:
df['REG_CONT'].value_counts()

1. Estado de Situación Financiera o Estado de Resultados Integrales (PyG)    100
2. Libro de registro de operaciones                                           71
3. Otro tipo de cuentas                                                       67
4. No lleva registros                                                         62
Name: REG_CONT, dtype: int64

In [121]:
df['REG_CONT'] = df['REG_CONT'].map(dict(zip(rel_reg_con['REG_CONT'], rel_reg_con['Codigo'])))

In [122]:
df['REG_CONT'].value_counts()

1    100
2     71
3     67
4     62
Name: REG_CONT, dtype: int64

### Antigüedad

In [123]:
df['ANIOS_OPERACION'] = 2024 - df['IDAIO']

### Ubicación

In [125]:
df['UBICA'] = df['UBICA'].map(dict(zip(rel_ubica['UBICA'], rel_ubica['Codigo'])))

In [126]:
df['UBICA'].value_counts()

3    78
4    75
1    74
2    73
Name: UBICA, dtype: int64

## Creación Variables

In [127]:
df['VA'] = df['ING_TOTAL'] - df['CYG']
df['C_V'] = df['CYG'] / (df['ING_TOTAL'] + 1)

## Selección Variables

In [128]:
var_estab = ['UID_ENCUESTA','ING_TOTAL','CYG','POTTOT','REMUNERACION_TOTAL','DPTO','CPAM','GRUPOS12','UBICA','IDRUT','CAMCOMER','REG_CONT','ANIOS_OPERACION','VA','C_V'] 

var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','ANOS_ESPACIO','HOMBRE']

In [129]:
df['UBICA'].value_counts()

3    78
4    75
1    74
2    73
Name: UBICA, dtype: int64

In [130]:
df_final = df[var_estab]

In [131]:
df_final.head()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,1,3.936199e+08,1.687142e+08,146,1.726303e+07,95,0,3,4,2,,2,1,2.249057e+08,0.428622
1,2,2.377089e+08,1.567058e+08,138,NaN,44,0,3,2,2,Cámara de Comercio,2,38,8.100306e+07,0.659234
2,3,7.148410e+08,1.504281e+08,157,NaN,44,0,3,2,1,,1,46,5.644129e+08,0.210436
3,4,4.067645e+08,1.144361e+08,136,NaN,81,0,3,2,2,NaN,2,5,2.923284e+08,0.281333
4,5,3.772557e+08,1.340726e+08,109,NaN,86,0,3,4,2,Cámara de Comercio,1,8,2.431831e+08,0.355389


## Filtrado de unidades

In [134]:
# Seleccionar unidades económicas ubicadas en vivienda o establecimiento fijo (1 y 3)
df_final = df_final[df_final['UBICA'].isin([1,3])]

In [139]:
# Departamentos con los que se entrenó el modelo
depto_emicron = [8, 13, 44, 47, 70, 20, 25, 11, 18, 41, 15, 68, 54, 76, 52, 19, 17, 63, 73, 66,  5, 23, 27, 50, 88]
df_final = df_final[df_final['DPTO'].isin(depto_emicron)]

In [177]:
#(df_final['CAMCOMER']==" ").value_counts()
df_final['CAMCOMER'].fillna(0, inplace = True)
df_final.loc[df_final['CAMCOMER']==" ",'CAMCOMER'] = 0

C:\Users\carlo\AppData\Local\Temp\ipykernel_10900\743499618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['CAMCOMER'].fillna(0, inplace = True)


In [178]:
# Descripción variables nulas
df_final.isna().sum()

UID_ENCUESTA            0
ING_TOTAL               0
CYG                     0
POTTOT                  0
REMUNERACION_TOTAL    105
DPTO                    0
CPAM                    0
GRUPOS12                0
UBICA                   0
IDRUT                   0
CAMCOMER                0
REG_CONT                0
ANIOS_OPERACION         0
VA                      0
C_V                     0
dtype: int64

In [142]:
# Descripción variables no responde
(df_final==99).sum()

UID_ENCUESTA          0
ING_TOTAL             0
CYG                   0
POTTOT                1
REMUNERACION_TOTAL    0
DPTO                  0
CPAM                  0
GRUPOS12              0
UBICA                 0
IDRUT                 0
CAMCOMER              0
REG_CONT              0
ANIOS_OPERACION       0
VA                    0
C_V                   0
dtype: int64

In [445]:
id_ig = df_final['UID_ENCUESTA'][((df_final.isna()) | (df_final==99)).any(axis=1)]
# df_final = df_final[~(df_final['UID_ENCUESTA'].isin(id_ig))]

In [446]:
df_final['UID_ENCUESTA']

6        7
7        8
8        9
10      11
11      12
      ... 
288    289
291    292
292    293
293    294
299    300
Name: UID_ENCUESTA, Length: 107, dtype: int64

In [447]:
len(id_ig)

106

In [448]:
df_final.head(14)

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
6,7,4.576379e+08,1.608479e+08,145,NaN,88,1,3,1,1,NaN,2,43,2.967899e+08,0.351474
7,8,2.889470e+08,8.189908e+07,145,NaN,20,0,3,1,2,Cámara de Comercio,2,17,2.070480e+08,0.283440
8,9,3.867773e+08,1.630674e+08,153,NaN,18,0,3,3,2,,3,5,2.237099e+08,0.421605
10,11,4.639375e+08,3.474247e+08,179,NaN,20,0,3,1,2,NaN,4,60,1.165128e+08,0.748861
11,12,4.967122e+08,1.548614e+08,142,NaN,44,0,3,1,1,NaN,1,29,3.418508e+08,0.311773
12,13,3.549875e+08,1.276670e+08,138,NaN,27,0,3,1,2,,4,21,2.273205e+08,0.359638
14,15,3.811541e+08,1.678021e+08,120,NaN,17,0,3,1,1,NaN,1,31,2.133520e+08,0.440247
17,18,2.742435e+08,1.357881e+08,137,NaN,17,0,3,1,2,NaN,3,4,1.384553e+08,0.495137
21,22,1.813283e+08,3.247078e+08,89,NaN,18,0,3,1,2,NaN,1,21,-1.433795e+08,1.790718
22,23,1.067330e+08,2.608031e+08,117,NaN,41,0,3,3,1,NaN,2,7,-1.540700e+08,2.443508


In [449]:
df_final['UID_ENCUESTA'][(df_final['UID_ENCUESTA'].isin(ids))]

25    26
Name: UID_ENCUESTA, dtype: int64

# Vendedores de calle: Simulación

In [2]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Despliegue prueba\Vcalle/"
dicc_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data/"

In [3]:
#### Lectura de archivos

#df_id: Módulo de identificación  
df = pd.read_csv(data_path+"Crudo/DB_SIMULADA_VDC.csv", sep=';',header = 0)

In [94]:
df['ACTIVIDAD'].isna().value_counts()
#df['USADO'].value_counts()
#df['NUEVOS'].value_counts()
#df['SERVICIO'].value_counts()
#df['PRODUCTO'].value_counts()
#pd.crosstab(df['ACTIVIDAD'],df['USADO'])

False    540166
True      59834
Name: ACTIVIDAD, dtype: int64

In [26]:
df['NUEVOS'].value_counts()

3. Carnes (incluye aves de corral), productos cárnicos, pescados y productos de mar    45564
6. Productos textiles, prendas de vestir y sus accesorios                              45219
9. Enseres y aparatos de uso doméstico, paraguas, entre otros                          45168
11. Desperdicios, desechos y chatarra                                                  45168
4. Leche, productos lácteos y huevos                                                   45152
2. Productos agrícolas (frutas y verduras)                                             45135
8. Recuerdos, artesanías y artículos religiosos (incluso velas)                        44990
5. Otros alimentos, bebidas y tabaco                                                   44932
10. Libros, periódicos, materiales y artículos de papelería y escritorio               44928
7. Todo tipo de calzado y artículos de cuero y sucedáneos del cuero                    44906
1. Productos de confitería o dulcería, caramelos, turrones, bocadillos

In [4]:
#df['CIIU_4'] = 1789
#df.to_csv(data_path+"Crudo/DB_SIMULADA_VDC (2).csv", sep=';', index=False)

# Diccionario
g4_dictionary ={"1. Venta de mercancías" : 3, "2. Prestación de servicios" : 4, "3. Elaboración o fabricación productos" : 2}

# Creación GRUPOS4
df['GRUPOS4'] = df['ACTIVIDAD'].map(g4_dictionary)

df.loc[df['USADO']=="1. Venta de mercancías",['GRUPOS4']] = 3
df.loc[df['USADO']=="2. Prestación de servicios",['GRUPOS4']] = 4
df.loc[df['USADO']=="3. Elaboración o fabricación productos",['GRUPOS4']] = 2


# Creación CPAM Ciudad Principal y Área Metropolitana
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,
        8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,
        54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	
        70001,	88001]
df['COD_MPIO'] = pd.to_numeric(df['COD_MPIO'], errors= 'coerce')
df['CPAM'] = (df['COD_MPIO'].isin(cpam) & df['CLASE']==1).apply(lambda x: int(x))

# Arreglar variable de Sexo
df.loc[~((df['SEXO']=="1. Hombre") | (df['SEXO']=="2. Mujer")),'SEXO'] = df.loc[~((df['SEXO']=="1. Hombre") | (df['SEXO']=="2. Mujer")),'FECHA_NAC']

# Arreglar variable ACTIVIDAD
df.loc[df['ACTIVIDAD'].isna(),'ACTIVIDAD'] = df.loc[df['ACTIVIDAD'].isna(),'USADO']

# Arreglar variable USADO
df.loc[~((df['USADO']=="2. No") | (df['USADO']=="1. Sí")),'USADO'] = df.loc[~((df['USADO']=="2. No") | (df['USADO']=="1. Sí")),'NUEVOS']

# Arreglar variable NUEVOS
df.loc[((df['NUEVOS']=="2. No") | (df['NUEVOS']=="1. Sí")),'NUEVOS'] = df.loc[((df['NUEVOS']=="2. No") | (df['NUEVOS']=="1. Sí")),'SERVICIO']


# Arreglar variable SERVICIO
df.loc[~(df['OBS_IV'].isna()),'SERVICIO'] = df.loc[~(df['OBS_IV'].isna()),'PRODUCTO']

# Arreglar variable PRODUCTO
df.loc[~(df['OBS_IV'].isna()),'PRODUCTO'] = df.loc[~(df['OBS_IV'].isna()),'OTRA_ACTIVIDAD']

# Obtener sólo el número como llegará en el CENU
df['ACTIVIDAD'] = df['ACTIVIDAD'].apply(lambda x: x.split('.')[0])
df['USADO'] = df['USADO'].apply(lambda x: x.split('.')[0])
df['NUEVOS'] = df['NUEVOS'].apply(lambda x: x.split('.')[0])
df['SERVICIO'] = df['SERVICIO'].apply(lambda x: x.split('.')[0])
df['PRODUCTO'] = df['PRODUCTO'].apply(lambda x: x.split('.')[0])

# Convertir a número
df['ACTIVIDAD'] = df['ACTIVIDAD'].astype(str).astype(int)
df['USADO'] = df['USADO'].astype(str).astype(int)
df['NUEVOS'] = df['NUEVOS'].astype(str).astype(int)
df['SERVICIO'] = df['SERVICIO'].astype(str).astype(int)
df['PRODUCTO'] = df['PRODUCTO'].astype(str).astype(int)

# Agregar variable de ubicación geográfica
df['Inicio Coordinado GPS'] = "4.6477089,-74.0916305"

In [7]:
len(df['UID_ENCUESTA'].unique())

99766

In [129]:
#  df['SEXO'].value_counts().sum()

# df['ACTIVIDAD'].value_counts()
# df['USADO'].value_counts().sum()
df['NUEVOS'].value_counts()
# df['SERVICIO'].value_counts().sum()
# df['PRODUCTO'].value_counts()
#pd.crosstab(df['ACTIVIDAD'],df['USADO'])

3     50600
6     50277
11    50164
9     50161
8     50034
4     49994
10    49966
2     49938
5     49894
7     49892
1     49647
12    49433
Name: NUEVOS, dtype: int64

In [103]:
df[['ACTIVIDAD','USADO','NUEVOS','SERVICIO','PRODUCTO']].head(10)

,ACTIVIDAD,USADO,NUEVOS,SERVICIO,PRODUCTO
0,2. Prestación de servicios,1. Sí,"9. Enseres y aparatos de uso doméstico, paragu...",1. Expendio de comidas preparadas,5. Otro
1,2. Prestación de servicios,1. Sí,"1. Productos de confitería o dulcería, caramel...",5. Mantenimiento y reparación de bicicletas y ...,"2. Elaboración de bisutería, cadenas, manillas..."
2,3. Elaboración o fabricación productos,2. No,"6. Productos textiles, prendas de vestir y sus...",1. Expendio de comidas preparadas,1. Elaboración de joyas
3,1. Venta de mercancías,1. Sí,"3. Carnes (incluye aves de corral), productos ...",2. Recolección de desechos,3. Elaboración de artesanías
4,3. Elaboración o fabricación productos,1. Sí,2. Productos agrícolas (frutas y verduras),7. Cuidar carros,"4. Elaboración de bolsos, mochilas y similares"
5,2. Prestación de servicios,2. No,7. Cuidar carros,7. Cuidar carros,3. Elaboración de artesanías
6,3. Elaboración o fabricación productos,1. Sí,12. Otros productos no incluidos en las otras ...,8. Otro,3. Elaboración de artesanías
7,1. Venta de mercancías,1. Sí,12. Otros productos no incluidos en las otras ...,3. Juegos de azar y apuestas,"2. Elaboración de bisutería, cadenas, manillas..."
8,1. Venta de mercancías,1. Sí,"9. Enseres y aparatos de uso doméstico, paragu...",8. Otro,"4. Elaboración de bolsos, mochilas y similares"
9,3. Elaboración o fabricación productos,1. Sí,"8. Recuerdos, artesanías y artículos religioso...",2. Recolección de desechos,5. Otro


In [134]:
df['ACTIVIDAD'].astype(str).astype(int)

0         2
1         2
2         3
3         1
4         3
         ..
599995    3
599996    1
599997    2
599998    1
599999    2
Name: ACTIVIDAD, Length: 600000, dtype: int32

In [139]:
#data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Despliegue prueba\Vcalle/"
#dicc_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data/"

#df = pd.read_csv(data_path + "Crudo/DB_SIMULADA_VDC (2).csv", sep = ';')

In [161]:
# Debido a que las estadísticas descriptivas de las variables numéricas distan de las que se observan en la base de emicron
# ya que el promedio diario de ventas, gastos y costos en los vendedores de calle resulta ser muy elevado, se considerará reemplazar
# los valores simluados con los de EMICRON para poder tener variabilidad en los resultados y que el modelo no indique como anómalos
# todos los registros. 
df[['VENTAS','GASTO_TOT','COSTOS','TRABAJSOC','TRABAJPAGO','TRABAJFAMI']].describe()

,VENTAS,GASTO_TOT,COSTOS,TRABAJPAGO,TRABAJFAMI
count,6.000000e+05,6.000000e+05,600000.000000,600000.000000,600000.000000
mean,3.559329e+06,3.000751e+06,416199.121991,1.950468,1.950912
std,2.756727e+06,6.427683e+05,296220.028492,1.013158,1.012813
min,7.328244e+00,2.663808e+05,0.000000,0.000000,0.000000
25%,1.342548e+06,2.577914e+06,157217.429325,1.000000,1.000000
50%,2.638366e+06,3.022171e+06,395857.846200,2.000000,2.000000
75%,5.576762e+06,3.447280e+06,657705.800325,2.000000,2.000000
max,9.999959e+06,4.875524e+06,999998.967900,20.000000,20.000000


In [162]:
# Abrir información de vendedores de calle: EMICRON

data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data"
emicron = pd.read_csv(data_path + "\EMICRON_VF.csv",header = 0)

var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','CYG','COD_DEPTO','CPAM','GRUPOS12','ANOS_ESPACIO','HOMBRE']
var_gral = ['id','CIIU_3','fex','P2760','VA','C_V','k_words','mencion']

emicron_mov = emicron[emicron['UBICA']== 4]
emicron_mov = emicron_mov[var_mov+var_gral+['GRUPOS4','COD_MPIO']]
emicron_mov['CPAM'] = emicron_mov['CPAM'].astype(int)
emicron_mov['COD_DEPTO'] = emicron_mov['COD_DEPTO'].astype(int)
emicron_mov['GRUPOS12'] = emicron_mov['GRUPOS12'].astype(int)

emicron_mov = emicron_mov.reset_index(drop=True)
emicron_mov.shape

C:\Users\carlo\AppData\Local\Temp\ipykernel_23356\3193646247.py:4: DtypeWarning: Columns (59,60,102,104,106,119,132,133,134,135,136,137,140,141,143,157,363,377,388,393,401,420,485,502,503,506,524,525,528,529) have mixed types. Specify dtype option on import or set low_memory=False.
  emicron = pd.read_csv(data_path + "\EMICRON_VF.csv",header = 0)


(17812, 19)

In [30]:
emicron['CIIU_3']

0        5611.0
1        4722.0
2        5613.0
3        5611.0
4        4723.0
          ...  
64476    5619.0
64477    9512.0
64478    1081.0
64479    9521.0
64480    5619.0
Name: CIIU_3, Length: 64481, dtype: float64

In [163]:
emicron_mov_f = emicron_mov.copy()
emicron_mov_f['VENTAS'] = emicron_mov_f['VENTAS_MES_ANTERIOR'] / 30
emicron_mov_f['GASTO_TOT'] = emicron_mov_f['CYG'] / 60
emicron_mov_f['COSTOS'] = emicron_mov_f['CYG'] / 60

emicron_mov_f['TRABAJSOC'] = emicron_mov_f['POTTOT']
emicron_mov_f['TRABAJPAGO'] = 0
emicron_mov_f['TRABAJFAMI'] = 0

# Diccionario
sex_dic ={1 : "1. Hombre", 0: "2. Mujer"}

# Creación GRUPOS4
emicron_mov_f['SEXO'] = emicron_mov_f['HOMBRE'].map(sex_dic)

# Renombrar columna
emicron_mov_f.rename(columns={'COD_DEPTO': 'COD_DPTO', 'CIIU_3':'CIIU_4'}, inplace=True)

In [164]:
emicron_mov_f.head()

,id,VENTAS_MES_ANTERIOR,POTTOT,CYG,COD_DPTO,CPAM,GRUPOS12,ANOS_ESPACIO,HOMBRE,id,...,mencion,GRUPOS4,COD_MPIO,VENTAS,GASTO_TOT,COSTOS,TRABAJSOC,TRABAJPAGO,TRABAJFAMI,SEXO
0,212002612,1200000.0,1.0,420000.0,8,0,5,1.0,1.0,212002612,...,False,3,b'433',40000.000000,7000.000000,7000.000000,1.0,0,0,1. Hombre
1,212005312,3600000.0,1.0,2690000.0,8,0,5,5.0,1.0,212005312,...,NaN,3,b'433',120000.000000,44833.333333,44833.333333,1.0,0,0,1. Hombre
2,212017711,7000000.0,2.0,4919000.0,8,0,3,5.0,1.0,212017711,...,NaN,2,b'078',233333.333333,81983.333333,81983.333333,2.0,0,0,1. Hombre
3,212020112,0.0,1.0,0.0,8,0,5,2.0,0.0,212020112,...,False,3,b'436',0.000000,0.000000,0.000000,1.0,0,0,2. Mujer
4,212020612,9000000.0,1.0,7915000.0,8,0,5,5.0,0.0,212020612,...,NaN,3,b'436',300000.000000,131916.666667,131916.666667,1.0,0,0,2. Mujer


In [165]:
df_b = df.drop(['VENTAS', 'GASTO_TOT', 'COSTOS', 'TRABAJSOC', 'TRABAJPAGO','TRABAJFAMI'], axis=1)

### Crear indice en emicron
emicron_mov_f['index_emicron'] = emicron_mov_f.groupby(['COD_DPTO','CPAM','SEXO','GRUPOS4']).cumcount() + 1

## Obtener el valor máximo del índice para cada combinación
max_indices = emicron_mov_f.groupby(['COD_DPTO','CPAM','SEXO','GRUPOS4'])['index_emicron'].max().reset_index()
max_indices.rename(columns={'index_emicron': 'max_index'}, inplace=True)

## Unir el valor máximo al df original para saber hasta dónde generar el índice aleatorio
df_b = pd.merge(df_b, max_indices, on=['COD_DPTO','CPAM','SEXO','GRUPOS4'], how='left')

## Generar un número aleatorio en df
df_b['index_emicron'] = df_b.apply(lambda row: np.random.randint(1, row['max_index'] + 1) if not pd.isna(row['max_index']) else np.nan, axis=1)


final_simulada = pd.merge(df_b,emicron_mov_f[['VENTAS', 'GASTO_TOT', 'COSTOS', 'TRABAJSOC', 'TRABAJPAGO','TRABAJFAMI','COD_DPTO', 'CPAM', 'SEXO', 'GRUPOS4','CIIU_4','index_emicron']], on = ['COD_DPTO','CPAM','SEXO','GRUPOS4','index_emicron'], how = "left")

In [166]:
final_simulada.drop(['max_index', 'index_emicron','CPAM','GRUPOS4'], axis = 1, inplace = True)
final_simulada.rename(columns={'UID_ENCUESTA': 'RESPONSEID'}, inplace=True)

In [167]:
final_simulada['ACTIVIDAD'].value_counts().sum()

600000

In [195]:
final_simulada[0:200002].to_csv(data_path + "Crudo/DB_SIMULADA_1.csv", sep = ';', index = False)
final_simulada[200000:400002].to_csv(data_path + "Crudo/DB_SIMULADA_2.csv", sep = ';', index = False)
final_simulada[-200002:].to_csv(data_path + "Crudo/DB_SIMULADA_3.csv", sep = ';', index = False)

In [176]:
np.unique([final_simulada[0:200002]['RESPONSEID'].append(final_simulada[200000:400002])])

C:\Users\carlo\AppData\Local\Temp\ipykernel_23356\3758010369.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  np.unique([final_simulada[0:200002]['RESPONSEID'].append(final_simulada[200000:400002])])


TypeError: to_append should be a Series or list/tuple of Series, got DataFrame

In [185]:
len(np.unique(final_simulada[0:200002]['RESPONSEID'].append(final_simulada[200000:400002]['RESPONSEID'])))

C:\Users\carlo\AppData\Local\Temp\ipykernel_23356\2568445493.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  len(np.unique(final_simulada[0:200002]['RESPONSEID'].append(final_simulada[200000:400002]['RESPONSEID'])))


98193

In [191]:
final_simulada['RESPONSEID'].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
599995     True
599996     True
599997     True
599998     True
599999     True
Name: RESPONSEID, Length: 600000, dtype: bool

In [8]:
df['UID_ENCUESTA'].dtype

dtype('int64')